In [13]:
#Author: kannan / unsupervised - clustering of content & keywords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import metrics
import joblib

In [14]:
find_Opt_clusters=False

In [15]:
#Reading the extracted cotent
in_filepath='../data/output/SS_Extracted_content.xlsx'
out_filepath='../data/output/2_NLP_Clustering.xlsx'

df=pd.read_excel(in_filepath)
df.drop(['url','html'],axis=1,inplace=True)
df.head()

,Id,title,content
0,1,Americans Held in North Korea Ask U.S. for Ass...,"PYONGYANG, North Korea — North Korea gave fore..."
1,2,"Apple Partners With Visa, MasterCard, AmEx for...",According to new reports from Bloomberg and ot...
2,3,Aussie Football Players Dress Up as Rolf Harri...,SYDNEY — Two Australian football players are a...
3,4,Australia Government Buys Bombproof BMW Fleet ...,SYDNEY — The Australia Prime Minister has orde...
4,5,Author Compares Women in Traditional Marriages...,"SYDNEY — On Australian panel show Q&A, author ..."


In [16]:
#Vectorizing the text data
documents = df['content'].tolist()

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [17]:
if find_Opt_clusters:
    #optimum number of clusters
    #Identifying optimum number of clusters\
    #https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    c=[]
    sil=[]
    for NUM_CLUSTERS in range(5,15):
        print(NUM_CLUSTERS)
        #kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
        kmeans = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
        kmeans.fit(X)
        labels = kmeans.labels_
        silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
        c.append(NUM_CLUSTERS)
        sil.append(silhouette_score)

    from pylab import *
    plot(c, sil)

    xlabel('#Clusters')
    ylabel('Silhouette Score')
    title('Number of Clusters')
    grid(True)
    show()

In [18]:
#Clustering
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [19]:
#writing the clustering model
joblib.dump(vectorizer, '../data/output/models/vectorizer_Tfidf.pkl')
joblib.dump(model, '../data/output/models/Clustering.pkl')

['../data/output/models/Clustering.pkl']

In [20]:
#Visualizing clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 cosby
 marvel
 wars
 star
 disney
 trailer
 movie
 allegations
 avengers
 awakens
Cluster 1:
 said
 state
 sony
 climate
 government
 2014
 president
 isis
 storm
 obama
Cluster 2:
 2014
 new
 said
 like
 just
 year
 time
 twitter
 people
 world
Cluster 3:
 app
 google
 company
 users
 facebook
 new
 mobile
 apps
 amazon
 social
Cluster 4:
 ebola
 health
 virus
 liberia
 disease
 said
 outbreak
 hospital
 africa
 duncan
Cluster 5:
 graduates
 michelle
 2020
 generations
 finish
 towers
 lego
 balloon
 unravels
 obama
Cluster 6:
 apple
 iphone
 ipad
 plus
 ios
 cook
 pay
 watch
 new
 event
Cluster 7:
 game
 summer
 nba
 kid
 attempt
 2011
 sports
 football
 just
 championship
Cluster 8:
 uber
 lyft
 drivers
 company
 taxi
 kalanick
 ride
 driver
 rides
 service
Cluster 9:
 police
 ferguson
 protesters
 hong
 kong
 officer
 brown
 officers
 protests
 2014


In [21]:
lables=model.labels_
df['cluster'] = lables.tolist()

df['cluster']='cluster_'+df['cluster'].apply(str)

df.head()

,Id,title,content,cluster
0,1,Americans Held in North Korea Ask U.S. for Ass...,"PYONGYANG, North Korea — North Korea gave fore...",cluster_1
1,2,"Apple Partners With Visa, MasterCard, AmEx for...",According to new reports from Bloomberg and ot...,cluster_6
2,3,Aussie Football Players Dress Up as Rolf Harri...,SYDNEY — Two Australian football players are a...,cluster_2
3,4,Australia Government Buys Bombproof BMW Fleet ...,SYDNEY — The Australia Prime Minister has orde...,cluster_1
4,5,Author Compares Women in Traditional Marriages...,"SYDNEY — On Australian panel show Q&A, author ...",cluster_2


In [22]:
#Get one hot encoding of cluster
one_hot = pd.get_dummies(df['cluster'])
# Drop column B as it is now encoded
df = df.drop('cluster',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df.head()

,Id,title,content,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9
0,1,Americans Held in North Korea Ask U.S. for Ass...,"PYONGYANG, North Korea — North Korea gave fore...",0,1,0,0,0,0,0,0,0,0
1,2,"Apple Partners With Visa, MasterCard, AmEx for...",According to new reports from Bloomberg and ot...,0,0,0,0,0,0,1,0,0,0
2,3,Aussie Football Players Dress Up as Rolf Harri...,SYDNEY — Two Australian football players are a...,0,0,1,0,0,0,0,0,0,0
3,4,Australia Government Buys Bombproof BMW Fleet ...,SYDNEY — The Australia Prime Minister has orde...,0,1,0,0,0,0,0,0,0,0
4,5,Author Compares Women in Traditional Marriages...,"SYDNEY — On Australian panel show Q&A, author ...",0,0,1,0,0,0,0,0,0,0


In [23]:
#New test example
test_str=df.loc[0,'content']
test_str[0:500]

'PYONGYANG, North Korea — North Korea gave foreign media access on Monday to three detained Americans who said they have been able to contact their families and — watched by officials as they spoke — called for Washington to send a high-ranking representative to negotiate for their freedom.\nJeffrey Fowle and Mathew Miller said they expect to face trial within a month. But they said they do not know what punishment they could face or what the specific charges against them are. Kenneth Bae, who alr'

In [24]:
df.drop(['title','content'],axis=1,inplace=True)
df.head()

,Id,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9
0,1,0,1,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,0
2,3,0,0,1,0,0,0,0,0,0,0
3,4,0,1,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0


In [25]:
#writing the output
#df.to_excel(out_filepath, index=False)

In [26]:
#Prediction
Y = vectorizer.transform([test_str])
prediction = model.predict(Y)
print(prediction)

[1]
